# London Passenger Mode Choice

In [ ]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

#import packages
import pandas as pd
from pycmtensor.dataset import Dataset
from pycmtensor.expressions import Beta, RandomDraws
from pycmtensor.models import MNL

lpmc = pd.read_csv("../data/lpmc.dat", sep="\t")
lpmc = lpmc[lpmc["travel_year"]==2015]
ds = Dataset(df=lpmc, choice="travel_mode")
ds.split(0.8)

# Beta parameters
asc_walk = Beta("asc_walk", 0.0, None, None, 1)
asc_cycle = Beta("asc_cycle", 0.0, None, None, 0)
asc_pt = Beta("asc_pt", 0.0, None, None, 0)
asc_drive = Beta("asc_drive", 0.0, None, None, 0)
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
s_time = Beta("s_time", 0.5, None, None, 0)
b_purpose = Beta("b_purpose", 0.0, None, None, 0)
b_licence = Beta("b_licence", 0.0, None, None, 0)
b_car_own = Beta("b_car_own", 0.0, None, None, 0)

rnd_time = RandomDraws('rnd_time', 'lognormal', 100)

U_walk = asc_walk + b_time * ds["dur_walking"] # + rnd_time(s_time) * db["dur_walking"]
U_cycle = asc_cycle + b_time * ds["dur_cycling"] 
U_pt = asc_pt + b_time * (ds["dur_pt_rail"] + ds["dur_pt_bus"] + ds["dur_pt_int"]) + b_cost * ds["cost_transit"]
U_drive = asc_drive + b_time * ds["dur_driving"] + b_licence * ds["driving_license"] + b_cost * (ds["cost_driving_fuel"] + ds["cost_driving_ccharge"])  + b_purpose * ds["purpose"]

U = [U_walk, U_cycle, U_pt, U_drive]
mymodel = MNL(ds, locals(), U)

## Estimate the models

In [ ]:
from pycmtensor.models import train
from pycmtensor.optimizers import SGD, Adam, AdaGrad, SQNBFGS, RMSProp
from pycmtensor.scheduler import StepLR, ConstantLR

mymodel.reset_values()
train(mymodel, ds, optimizer=Adam, batch_size=0, base_learning_rate=1., convergence_threshold=0.001, max_steps=2000, lr_scheduler=ConstantLR, lr_stepLR_drop_every=20, lr_stepLR_factor=0.99, BFGS_warmup=30)

## Display results

In [ ]:
pd.options.display.width = 99
print(mymodel.results.beta_statistics())
print(mymodel.results.model_statistics())
print(mymodel.results.benchmark())

## Correlation matrix

In [ ]:
print(mymodel.results.model_correlation_matrix())
print(mymodel.results.model_robust_correlation_matrix())

## Predictions

In [ ]:
prob = mymodel.predict(ds, return_probabilities=True)
print(pd.DataFrame(prob))

choice = mymodel.predict(ds, return_probabilities=False)
print(pd.DataFrame(choice))

## Elasticities

In [ ]:
elas = mymodel.elasticities(ds, wrt_choice=1)
print(pd.DataFrame(elas))